# Arabic Poetry Tashbeeh Classification
Train a binary classifier using AraBERT to detect tashbeeh in Arabic poetry lines.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score

2025-05-07 23:09:22.512159: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load Dataset

In [2]:
# Load the cleaned dataset
df = pd.read_csv("merged_poems_dataset_cleaned.csv")
df.head()

,النص,label
0,فبات على البرج المطل، كأنما يلاحظ منه فرقدا ثم...,1
1,غردت على فنن فأسعد شجوها ورق يجبن كما استجاب ا...,1
2,وإذا ما انبعثت أجري إليها كنت كالمبتغي مع الشم...,1
3,وكأنما أبدى لنا بمدامه وجماله صاع العزيز ويوسفا,1
4,فلا تسقطنكم بعدها آل مالك شرار أحاديث الرجال و...,0


## Split into Train, Validation, and Test Sets

In [3]:
# Split into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.95, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.98, stratify=temp_df['label'], random_state=42)

## Load Tokenizer

In [4]:
model_name = "aubmindlab/bert-base-arabertv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Tokenize Text and Prepare Datasets

In [5]:
def tokenize_example(example):
    return tokenizer(example['النص'], truncation=True, padding="max_length")

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds = Dataset.from_pandas(val_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))

train_ds = train_ds.map(tokenize_example, batched=True)
val_ds = val_ds.map(tokenize_example, batched=True)
test_ds = test_ds.map(tokenize_example, batched=True)

train_ds = train_ds.remove_columns(['النص'])
val_ds = val_ds.remove_columns(['النص'])
test_ds = test_ds.remove_columns(['النص'])

Map:   0%|          | 0/2926 [00:00<?, ? examples/s]

Map:   0%|          | 0/1112 [00:00<?, ? examples/s]

Map:   0%|          | 0/54490 [00:00<?, ? examples/s]

## Load Pre-trained AraBERT Model

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define Accuracy Metric

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

## Training Configuration

In [8]:
training_args = TrainingArguments(
    output_dir="./cls_model",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

## Initialize Trainer

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics
)

/home/faisal/anaconda3/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Train Model

In [10]:
trainer.train()

  0%|          | 0/3660 [00:00<?, ?it/s]

{'loss': 0.445, 'grad_norm': 16.333595275878906, 'learning_rate': 4e-05, 'epoch': 1.0}


  0%|          | 0/278 [00:00<?, ?it/s]

{'eval_loss': 0.39106038212776184, 'eval_accuracy': 0.927158273381295, 'eval_runtime': 59.6459, 'eval_samples_per_second': 18.643, 'eval_steps_per_second': 4.661, 'epoch': 1.0}
{'loss': 0.3089, 'grad_norm': 0.11239004135131836, 'learning_rate': 3e-05, 'epoch': 2.0}


  0%|          | 0/278 [00:00<?, ?it/s]

...

  0%|          | 0/278 [00:00<?, ?it/s]

{'eval_loss': 0.22894181311130524, 'eval_accuracy': 0.9595323741007195, 'eval_runtime': 60.5986, 'eval_samples_per_second': 18.35, 'eval_steps_per_second': 4.588, 'epoch': 4.0}
{'loss': 0.1326, 'grad_norm': 0.04942960664629936, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/278 [00:00<?, ?it/s]

{'eval_loss': 0.27220967411994934, 'eval_accuracy': 0.9532374100719424, 'eval_runtime': 59.5574, 'eval_samples_per_second': 18.671, 'eval_steps_per_second': 4.668, 'epoch': 5.0}
{'train_runtime': 2590.0846, 'train_samples_per_second': 5.648, 'train_steps_per_second': 1.413, 'train_loss': 0.2595008183046768, 'epoch': 5.0}


TrainOutput(global_step=3660, training_loss=0.2595008183046768, metrics={'train_runtime': 2590.0846, 'train_samples_per_second': 5.648, 'train_steps_per_second': 1.413, 'train_loss': 0.2595008183046768, 'epoch': 5.0})

## Evaluate on Test Set

In [11]:
results = trainer.evaluate(test_ds)
print("📊 Test Results:", results)

  0%|          | 0/13623 [00:00<?, ?it/s]

📊 Test Results: {'eval_loss': 0.2746935486793518, 'eval_accuracy': 0.9515690952468343, 'eval_runtime': 2943.9199, 'eval_samples_per_second': 18.509, 'eval_steps_per_second': 4.628, 'epoch': 5.0}


: 